# facenet-pytorch package 사용



0. pip install facenet-pytorch


1. loading all video frame


2. finding all faces


3. calculating face embeddings at over 30 frames per second




## MTCMM 모듈


work directly with images represented as **numpy arrays.**


acieves higher performance when reading video frames with either **cv2.VideoCapture or decord.VideoReader**


A number of additional enhancements have been added to improve detection efficiency


 "How to detect every face in every frame in every video"

In [5]:
import sys, os
sys.path.insert(0,'/kaggle/working/reader/python')

from facenet_pytorch import MTCNN
import torch
import cupy
from decord import VideoReader, gpu
import glob
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

ModuleNotFoundError: No module named 'cupy'

cupy 설치하는거 잘 안됨


GPU 가 필요조건이라는데 이게 없어서 그런가...?


anaconda 로는 될까...? **pip install conda**

anaconda 설치가 안돼서 pyenv 라는 homebrew 환경의 솔루션 사용

**brew install pyenv**



아나콘다 또 다시 설치중~~ 이번엔 좀 돼라




## FastMTCNN 클래스 구현


MTCNN 은 face detection -> N개의 프레임마다 수행, 모든 프레임에 적용됨



In [ ]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, *args, **kwargs):
        """Constructor for FastMTCNN class.
        
        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.
        
        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        probs_out = []
        frame_index = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box, prob in zip(boxes[box_ind], probs[box_ind]):
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]].copy())
                probs_out.append(prob)
                frame_index.append(i)
                
        
        return faces, probs, frame_index

## Face Detector 정의

In [ ]:
fast_mtcnn = FastMTCNN(
    stride=10,
    margin=20,
    factor=0.6,
    keep_all=True,
    device=device,
    thresholds=[0.6, 0.7, 0.98]
)

## 모든 비디오에 대해

In [ ]:
%%time

def mean_detection_prob(prob):
    cnt_p = 0
    sum_p = 0
    for p in prob:
        for pp in p:
            if pp is not None:
                cnt_p += 1
                sum_p += pp
    return sum_p / cnt_p


def get_frames(filename, batch_size=30):
    v_cap = VideoReader(filename, ctx=gpu())
    v_len = len(v_cap)

    frames = []
    for i in range(0, v_len, batch_size):
        batch = v_cap.get_batch(range(i, min(i + batch_size, v_len - 1))).asnumpy()
        frames.extend(batch.copy())
    
    frames = np.array(frames)
    
    del v_cap, v_len, batch
    
    return frames


filenames = glob.glob('/kaggle/input/deepfake-detection-challenge/test_videos/*.mp4')

num_faces = 0
probs = []
indexes = []
pbar = tqdm(filenames)
for filename in pbar:
    frames = get_frames(filename)

    faces, prob, index = fast_mtcnn(frames)        
    probs.append(mean_detection_prob(prob))

    num_faces += len(faces)
    pbar.set_description(f'Faces found: {num_faces}')

    del frames

In [ ]:
probs = np.asarray(probs)
probs = np.clip((1 - probs) ** (1 / 6) * 1.7, 0.0, 1.0)
plt.hist(probs, 40);

filenames = [os.path.basename(f) for f in filenames]

submission = pd.DataFrame({'filename': filenames, 'label': probs})
submission.to_csv('submission.csv', index=False)
submission

https://www.kaggle.com/timesler/facenet-pytorch-decord-process-every-frame 